
## Objectifs
##### 1 Lire et traiter des données d'entrée avec [PANDAS](https://pandas.pydata.org/)

##### 2 Représenter la distribution empirique des "fold-change" (**FC**) vs Précision, aka: volcano-plot
    
##### 3 Effectuer une analyse de sur-représentation en termes GO


## Ressources : Rappels et illustration de l'analyse de représentation de termes GO
###### [TP de MADP](https://github.com/glaunay/tp-proteomics#3-obtention-des-param%C3%A8tres-du-mod%C3%A8le)
###### [Fiche Bioconductor](https://www.bioconductor.org/help/course-materials/2015/SeattleApr2015/E_GeneSetEnrichment.html)



### Préparation de  l'environnement

##### Please `pip install -r requirements.txt` first

##### Directory Configuration

* `workDir` points to the Git project, holds the *tsv* file
* `libDir` points to Git project subdirectory hosting python library with files names `go.py  stat_utils.py  uniprot.py`
* `dataDir` points to the data folder with uniprot and GO files

In [1]:
import sys, os
workDir = f"{os.getcwd()}/.."
libDir = f"{workDir}/lib"
dataDir= f"{workDir}/data" 

sys.path.append(libDir)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas
import numpy as np

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt

## Jeux de données
* Jeux de données de protéomique quantitative au format TSV
* Ontologie GO au format [owl](http://www.obofoundry.org/ontology/go.html)
* Entrées Uniprot au format XML
    * Protéines de l'étude
    * Protéome d'E.Coli complet
    
<hr style="border:1px solid gray"> </hr>

### Lecture des valeurs experimentales
Charger ```TCL_wt1.tsv``` dans une [dataframe pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).
<br/><span style="color:firebrick;font-weight:bold"> $\rhd$ 
Retirer les lignes présentant des aberrations numériques
</span>
<br/><span style="color:firebrick;font-weight:bold">$\rhd$ Attention aux types des colonnes !</span>

In [ ]:
dico = {"Accesion" : str, 
        "Description" : str, 
        "Gene Symbol" : str, 
        "Corrected Abundance Ratio (1.53)" : float,
        "Log2 Corrected Abundance Ratio" : float,
        "Abundance Ratio Adj. P-Value: (127. T3 Tc WT) / (126. T0 WT)" : float,
        "LOG10 Adj.P-val" : float}

df = pandas.read_csv("../TCL_wt1.tsv", header = 0, delimiter = "\t", dtype = dico, na_values = ["#VALEUR!", "nan"])
df = df.dropna()
df.head(3)

##### 1. Un premier volcano-plot

Ce type de figure représente pour chaque protéine en abscisse le taux d'abondance et en ordonnée la précision de la mesure.

<span style="color:firebrick;font-weight:bold">$\rhd$ 1. Réaliser un scatter plot matplotlib avec</span>
<br/><span style="color:firebrick;font-weight:bold">- en x, `Log2 Corrected Abundance Ratio`</span>
<br/><span style="color:firebrick;font-weight:bold">-  en y, `LOG10 Adj.P-val`</span>

<span style="color:firebrick;font-weight:bold">$\rhd$Vous légenderez les axes, attention `LOG10 Adj.P-val` est en fait `-LOG10 Adj.P-val`, regardez bien.</span>

##### 2. Réaliser des versions évoluées du graphique précédent
L'objectif est de mettre l'accent sur la région **-Log10 adj Pvalue > 4** et **Log2 corrected abundance ratio > 0**.
<br/>Pour cela vous revisiterez le scatter plot précedent au travers des versions suivantes:
<br/><span style="color:firebrick;font-weight:bold"> $\rhd$ première version : un rectangle de la couleur de votre choix matérialisera la région</span>
<br/><span style="color:firebrick;font-weight:bold"> $\rhd$ deuxième version  : une couleur de votre choix représentera les points de la région</span>
<br/><span style="color:firebrick;font-weight:bold"> $\rhd$ troisème version (optionnel): Les identifiants uniprot remplaceront les points des protéines de la région</span>

In [ ]:
x = df["Log2 Corrected Abundance Ratio"]
y = df["LOG10 Adj.P-val"]

fig, ax = plt.subplots()

ax.scatter(x, y)
ax.set_xlabel("Log2 Ratio")
ax.set_ylabel("Log10 P-val")

# Version 1
import matplotlib.patches as patches

x_ancre = min(x[x>0])
y_ancre = min(y[y>4])

rect = patches.Rectangle((x_ancre, y_ancre), max(x)+0.2, 3, linewidth=1, edgecolor='r', facecolor='none')
ax.add_patch(rect)

In [ ]:
# Version 2

x = df["Log2 Corrected Abundance Ratio"]
y = df["LOG10 Adj.P-val"]

fig, ax = plt.subplots()

ax.scatter(x, y)
ax.set_xlabel("Log2 Ratio")
ax.set_ylabel("Log10 P-val")

x_2 = []
y_2 = []

for i in range(len(x)):
    if list(x)[i] > 0 and list(y)[i] > 4:
        x_2.append(list(x)[i])
        y_2.append(list(y)[i])

ax.scatter(x_2, y_2, color = "r")

In [ ]:
# Version 3
x = df["Log2 Corrected Abundance Ratio"]
y = df["LOG10 Adj.P-val"]
label = df["Accession"]

fig, ax = plt.subplots(figsize = (10,6))

x_not = []
y_not = []
x_yes = []
y_yes = []
label_test = []

for i in range(len(x)):
    if list(x)[i] < 0 or list(y)[i] < 4:
        x_not.append(list(x)[i])
        y_not.append(list(y)[i])
    else:
        label_test.append(list(label)[i])
        x_yes.append(list(x)[i])
        y_yes.append(list(y)[i])
        


ax.scatter(x_not, y_not)
ax.set_xlabel("Log2 Ratio")
ax.set_ylabel("Log10 P-val")

for i, txt in enumerate(label_test):
    ax.annotate(txt, (x_yes[i], y_yes[i]))


#### Analyse ORA

##### Principes
Vous disposez d'un objet analyser permettant de réaliser une analyse de la sur-représentation (**ORA**) en termes GO parmi une liste de protéine d'intérêt.
La fréquence des termes GO dans le protéome totale est utilisée comme référence.

L'objet analyser s'instancie ainsi
```python
from stat_utils import GO_ORA_analyser
o = GO_ORA_analyser(f"{dataDir}/go.owl", f"{dataDir}/K12_proteome", f"{dataDir}/dataset")
```

Il fournit une méthode pour chaque catégorie de termes GO
<i>{ biological_process, molecular_function, cellular_component }</i>

Une éventuelle surreprésentation des termes GO de la catégorie <i>biological_process</i> est par exemple calculée parmi les protéines P29744 et P05706 de la façon suivante:

```python
goTerm_scores = o.biological_process(["P29744", "P05706"])
```
<span style="color:green;font-weight:bold">$\uparrow$ Reproduire cet exemple dans la cellule ci-dessous.$\downarrow$ </span>
</br><span style="color:firebrick;font-weight:bold"> $\rhd$ Inspecter l'objet retourné par la méthode o.biological_process, que voyez-vous ?</span>

In [ ]:
from stat_utils import GO_ORA_analyser
o = GO_ORA_analyser(f"{dataDir}\go.owl", f"{dataDir}\K12_proteome", f"{dataDir}\dataset")

In [ ]:
goTerm_scores = o.biological_process(["P29744", "P05706"])

In [ ]:
goTerm_scores

###### 2/ Analyser la représentation des termes GO parmi les protéines surabondantes de l'experience
<span style="color:firebrick;font-weight:bold"> $\rhd$ Vous devrez extraire les identifiants uniprot des protéines aux `Log2 Corrected Abundance Ratio` supérieurs au seuil alpha de 5%.</span>
<br/><span style="color:firebrick;font-weight:bold"> $\rhd$ Vous conduirez sur cette liste de protéines une analyse de sur-représentation en termes GO à l'aide d'un objet *GO_ORA_analyser*.</span>
<br/><span style="color:firebrick;font-weight:bold"> $\rhd$ Vous pouvez faire varier le seuil d'abondance autour de la valeur alpha.</span>
<br/><span style="color:firebrick;font-weight:bold"> $\rhd$ Vous traiterez les 3 catégories *biological_process / molecular_function /cellular_component*.</span>



Familiarisez-vous avec la structures des dictionnaires de résultas, vous pouvez [les sauvegarder au format json](https://docs.python.org/3/library/json.html).


In [ ]:
prot_log2 = df.query('`Log2 Corrected Abundance Ratio`>-0.096')['Accession']
prot_log2.shape
plog2 = prot_log2.to_list()

In [ ]:
o2 = GO_ORA_analyser(f"{dataDir}/go.owl", f"{dataDir}/K12_proteome", f"{dataDir}/dataset")

bioprocess_plog2 = o2.biological_process(plog2)

molfun_plog2 = o2.molecular_function(plog2)

cellcomp_plog2 = o2.cellular_component(plog2)

In [ ]:
molfun_plog2
cellcomp_plog2
bioprocess_plog2

In [ ]:
dbp_plog2 = list()
for i in bioprocess_plog2:
    dico_dat = {k:j for k, j in zip(["pval", "GO_term", "GO_num", "protein"], i)}
    dbp_plog2.append(dico_dat)
    

dmf_plog2 = list()
for i in molfun_plog2:
    dico_dat = {k:j for k, j in zip(["pval", "GO_term", "GO_num", "protein"], i)}
    dmf_plog2.append(dico_dat)
    
    
dcc_plog2 = list()
for i in cellcomp_plog2:
    dico_dat = {k:j for k, j in zip(["pval", "GO_term", "GO_num", "protein"], i)}
    dcc_plog2.append(dico_dat)

In [ ]:
dmf_plog2

In [ ]:
proteins_hits_GO_annot = GO_list(dmf_plog2, "molecular function")

proteins_hits_GO_annot

## Une première application: représentation riche du résultat de l'analyse


### Mise en forme "riche" des résultats grâce au notebook

Jupyter permet d'étendre le concept de la méthode **__str__()** afin de produire du contenu HTML.
 Ainsi, tout objet impémentant une méthode **_repr_html_()** sera affiché, via son rendu HTML, dans la cellule d'un notebook.

```python
class Point():
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
    def _repr_html_(self):
        htmlString = "<table><thead><tr><td>Abscisse</td><td>Ordonnée</td><td>Cote</td></tr></thead>"
        htmlString += f"<tbody><tr><td>{self.x}</td><td>{self.y}</td><td>{self.z}</td></tr></tbody></table>"
        return htmlString
Point(2,3,4)
```
Affichera

![caption](../figs/reprhtml.png)


### Intégration des résultats à la visualisation du notebook 

En vous inspirant de la fonction ci-dessus, implémenter une classe Rich_GO_ORA dont la méthode [`_repr_html_`](https://ipython.readthedocs.io/en/stable/config/integrating.html) permettra un affichage plus lisible d'une des analyses `biological_process / molecular_function /cellular_component`

Un exemple possible est,
![caption](../figs/_repr_html_.png)

(Bonus): clicker sur le nom du terme GO envoie vers sa description.
Pour cela, afficher le nom du pathway dans une balise hyperlien pour permettre d'acceder à la page de description du terme GO. 

Les termes GO sont accessibles aux URL de la forme
`http://amigo.geneontology.org/amigo/term/GO:GO_NUMBER`.




In [ ]:
class GO():
    def __init__(self, dico):
        self.pval = dico["pval"]
        self.GO_term = dico["GO_term"]
        self.GO_num = dico["GO_num"]
        self.protein = dico["protein"]
        
    def _repr_html_(self):
        link = f"<a href='http://amigo.geneontology.org/amigo/term/{self.GO_num}'>{self.GO_num}</a>"
        htmlString = f"<tbody><tr><td>{np.round(self.pval, 8)}</td><td>{self.GO_term}</td><td>{link}</td><td>{len(self.protein)}</td></tr></tbody>"
        return htmlString
    

class GO_list():  #collection of GO instance 
    def __init__(self, a_list, desc):
        self.list = [GO(i) for i in a_list]
        self.desc = desc
        
    def _repr_html_(self):
        htmlString = f"<h1>{self.desc}</h1><table><thead><tr><td>pval</td><td>{self.desc}</td><td>go_num</td><td>number</td></tr></thead>"
        for i in self.list:
            htmlString += i._repr_html_()
        htmlString += "</table>"
        return htmlString
    
    def get_accession(self):
        return {i.GO_term:i.protein for i in self.list}

In [ ]:
proteins_hits_GO_annot = GO_list(dmf_plog2, "molecular function")

proteins_hits_GO_annot

## Une deuxième application: volcano plot améliorés
###### PANDAS
En choisissant comme seuil la valeur `alpha` du TP précédent, veuillez extraires les identifiants uniprot des protéines sur-abondantes (rappel les valeurs d'abondance sont celles de la colonne `Log2 Corrected Abundance Ratio`).

Vous devrez avoir dans des listes distinctes:
* identifiants uniprot
* `Log2 Corrected Abundance Ratio`
* `LOG10 Adj.P-val'`

###### Représentation graphiques
* Pour la catégorie de termes GO (biological_process / molecular_function /cellular_component) de votre choix
    * Générer une grille de 4 graphiques 
    * Dans chaque graphique colorez dans une couleur différentes les protéines porteuses de 4 termes de GO que vous estimez les plus pertinents 
    * Donnez le nom de la catégorie générale à la grille
    * Donnez le nom du terme GO représenté dans chaque graphique avec une couleur de titre cohérente.
    

In [ ]:
subset_good = df[df["Log2 Corrected Abundance Ratio"] > -0.096]

prot_ration = subset_good["Log2 Corrected Abundance Ratio"].to_list()
pval = subset_good["LOG10 Adj.P-val"].to_list()
name = subset_good["Accession"].to_list()

In [ ]:
# Beaucoup d'overlap dans les noms des pathways.
# Certains pathways représentent la même fonction biologique --> Redondance

keep = GO_list(dbp_plog2 , "biological process")

keep.list = [keep.list[0]] + [keep.list[4]] + [keep.list[8]] + [keep.list[10]]
keep

In [ ]:
data = keep.get_accession()

f_4, a_4 = plt.subplots(ncols=2, nrows=2, figsize=(8, 8))
f_4.tight_layout(pad=5.0)

for idx, items in zip(range(4), data.items()):
    rw, cl = idx % 2, idx // 2
    title, acession_list = items
    
    # base scatterplot
    x, y = df["Log2 Corrected Abundance Ratio"], df["LOG10 Adj.P-val"]
    
    a_4[rw, cl].scatter(x, y, marker="+", alpha=0.5)
    a_4[rw, cl].set_xlabel("log2 Fold-Change")
    a_4[rw, cl].set_ylabel("-log10(p-value adj)")
    a_4[rw, cl].set_title(title)
    
    #add GO pathways protein in red
    x_c = df[df['Accession'].isin(acession_list)]["Log2 Corrected Abundance Ratio"]
    y_c = df[df['Accession'].isin(acession_list)]["LOG10 Adj.P-val"]
        
    a_4[rw, cl].scatter(x_c, y_c, c="red", label="")
    

    
## BONUS calc common accession in 4 GO term
accessions=[set(i) for i in data.values()]

common = accessions[0]
for i in accessions[1:]:
    common = common & i
    
common = list(common)
    
    
for idx in range(4):
    rw, cl = idx % 2, idx // 2
    
    x_com = df[df['Accession'].isin(common)]["Log2 Corrected Abundance Ratio"]
    y_com = df[df['Accession'].isin(common)]["LOG10 Adj.P-val"]
    
    a_4[rw, cl].scatter(x_com, y_com, c="green")

# Surprise

## Triangle de Sieprinski

In [ ]:
# Library
import math
from matplotlib.patches import Polygon

In [ ]:
# Definition des classes

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y


class Triangle:
    def __init__(self, p1, p2, p3):
        self.p1 = p1
        self.p2 = p2
        self.p3 = p3

    def __repr__(self):
        return(f"p1 : ({self.p1.x}, {self.p1.y}), p2 : ({self.p2.x}, {self.p2.y}), p3 : ({self.p3.x}, {self.p3.y})")

In [ ]:
# Definition des fonctions

def draw_triangle(p1, p2, p3, col = "black"):

    fig, ax = plt.subplots(figsize = [5,5])
    ax.plot([p1.x,p2.x,p3.x], [p1.y,p2.y,p3.y], color = col)
    ax.fill_between([p1.x,p2.x,p3.x], [p1.y,p2.y,p3.y], color = col)
    
    return fig, ax


def draw_triangle_recursif(ax, liste_triangle, n, max_iter, col = "white"):

    liste_triangle_temp = []
    if n < max_iter:

        for triangle in liste_triangle:

            # print(f"Triangle = {triangle}")

            p4 = Point((triangle.p1.x + triangle.p2.x) / 2, (triangle.p1.y + triangle.p2.y) /2)
            p5 = Point((triangle.p1.x + triangle.p3.x) / 2, (triangle.p1.y + triangle.p3.y) /2)
            p6 = Point((triangle.p2.x + triangle.p3.x) / 2, (triangle.p2.y + triangle.p3.y) /2)

            patche = Polygon([[p4.x, p4.y], [p5.x, p5.y], [p6.x, p6.y]], fill = True, color = col, linewidth = 0.00001)
            ax.add_patch(patche)
            
            # print(f"n : {n}, len(liste) = {len(liste_triangle)}")
            liste_triangle_temp.append(Triangle(triangle.p1, p4, p5))
            liste_triangle_temp.append(Triangle(p4, triangle.p2, p6))
            liste_triangle_temp.append(Triangle(p5, p6, triangle.p3))
    
        n += 1

        return(draw_triangle_recursif(ax, liste_triangle_temp, n, max_iter))

In [ ]:
p1 = Point(0,0)
p2 = Point(200,200)
p3 = Point(400,0)

test = Triangle(p1, p2, p3)

test2 = [test]

fig, ax = draw_triangle(p1,p2,p3)


n = 0


# Changer pour augmenter ou diminuer le nombre de récursion pour le triangle de sierpinski
# Attention, complixité = 3^n
max_iter = 7



draw_triangle_recursif(ax, test2, n, max_iter)


ax.text(0, 200, f"nombre d'itération = {max_iter}")
plt.show()